# Data Wrangling: `dplyr`

## observation $\equiv$ row, variable $\equiv$ column

## Datasets `nycflights13`

### Needed packages

In [ ]:
library(tidyverse)
library(nycflights13)

# Packages needed internally
library(knitr)
library(kableExtra)
library(IRdisplay)

### `nycflights13` package

- `nycflights13`: data related to all domestic flights departing from one of New York City's three main airports in 2013
  - Newark Liberty International (EWR)
  - John F. Kennedy International (JFK)
  - LaGuardia Airport (LGA)
- Contains five datasets saved in five data frames
  - `flights`: Information on all 336,776 flights
  - `airlines`: A table matching airline names and their two-letter International Air Transport Association (IATA) airline codes (a.k.a. carrier codes) for 16 airline companies
  - `planes`: Information about each of the 3,322 physical aircraft used
  - `weather`: Hourly meteorological data for each of the three NYC airports
    - Has 26,115 rows, roughly corresponding to the 365 $\times$ 24 $\times$ 3 = 26,280 possible hourly measurements one can observe at three locations over the course of a year
  - `airports`: Names, codes, and locations of the 1,458 domestic destinations

### Ways of exploring data frames

1. Using the `View()` function, which brings up built-in data viewer
1. Using the `glimpse()` function included in `dplyr`
1. Using the `kable()` function included in `knitr`

In [ ]:
glimpse(flights)

In [ ]:
kable(airlines)

### Identification and measurement variables

In [ ]:
glimpse(airports)

- `faa` and `name` are *identification variables*, variables that uniquely identify each observation
- The remaining variables (`lat`, `lon`, `alt`, `tz`, `dst`, `tzone`) are called *measurement* or *characteristic* variables: variables that describe properties of each observation 

- Sometimes a single variable might not be enough to uniquely identify each observation; combinations of variables might be needed 
- For organizational purposes it is considered good practice to have identification variables in the leftmost columns of the data frame.

## Introduction to `dplyr`

- Key functions of `dplyr`:
  1. `filter()` to pick out a subset of rows
  1. `summarize()` to generate one or more of its columns (variables) with a *summary statistic*: e.g. the median, the average, etc 
  1. `group_by()` to group its rows: assign different rows to be part of the same *group*
  1. `mutate()` its existing columns (variables) to create new ones
  1. `arrange()` its rows: e.g. sorting in ascending or descending order of some variables
  1. `join()` it with another data frame by matching along a "key" variable: merge these two data frames together

## The pipe operator: `%>%` 

- The pipe operator `%>%` allows us to combine multiple operations into a single sequential *chain* of actions
- Start with an example 
  - Say you would like to perform a sequence of operations on a data frame `x` using functions `action1()`, `action2()`, `action3()`, and `action4()`
    1. Take `x` *then*
    1. Use `x` as an input to a function `action1()` *then*
    1. Use the output of `action1(x)` as an input to a function `action2()` *then*
    1. Use the output of `action2(action1(x))` as an input to a function `action3()`
    1. Use the output of `action3(action2(action1(x)))` as an input to a function `action4()`
  - One way to achieve this sequence of operations is by using nesting parentheses as

In [ ]:
action4(action3(action2(action1(x))))

- This code is somehow hard to read 
- You can imagine that this will get progressively harder to read as the number of functions applied increases and arguments in each function increase as well
- This is where `%>%` comes in handy
  - `%>%` takes the output of one function and then "pipes" it to be the input of the next function
  - A helpful trick is to read `%>%` as "then" or "and then." 
  - For example, one obtain the same output as the sequence of functions as follows

In [ ]:
x %>% 
  action1() %>% 
  action2() %>% 
  action3() %>%
  action4()

- Read this sequence as:
  1. Take `x` *then*
  1. Use this output as the input to the next function `action1()` *then*
  1. Use this output as the input to the next function `action2()` *then*
  1. Use this output as the input to the next function `action3()` *then*
  1. Use this output as the input to the next function `action4()`

- While both approaches achieve the same goal, the latter is much more human-readable because one can clearly read the sequence of operations line-by-line 
 
- Throughout this introduction:
  1. The starting value `x` will be a data frame
  1. The sequence of functions `action1()`, `action2()`, `action3()`, and `action4()` will mostly be a sequence of any number of the six data wrangling verb-named key functions 
  1. The result will be the transformed/modified data frame
  
- Many more advanced data wrangling functions than just the key six ones listed

## `filter` rows 

![Diagram of `filter()` rows operation](./fig/filter.png)

- Focusing flights from New York City to Portland, Oregon
  - The `dest` destination code (or airport code) for Portland, Oregon is `"PDX"`
  - Run the following and look at the results to ensure that only flights heading to Portland are chosen

In [ ]:
portland_flights <- flights %>% 
    filter(dest == "PDX")
View(portland_flights)

- Note the order of the code
  - Take the `flights` data frame *then* `filter()` the data frame so that only those where the `dest` equals `"PDX"` are included
  - Test for equality using `==`, not `=`
  - One can use other operators `>`, `<`, `>=`, `<=`, `!=`
  - One can combine multiple criteria using operators `&`, `|`
- See these in action
  - Filter `flights` for all rows that departed from JFK *and* were heading to Burlington, Vermont (`"BTV"`) or Seattle, Washington (`"SEA"`) *and* departed in the months of October, November, or December:

In [ ]:
btv_sea_flights_fall <- flights %>% 
    filter(origin == "JFK" & (dest == "BTV" | dest == "SEA") & month >= 10)
View(btv_sea_flights_fall)

- One can skip the use of `&` and just separate the conditions with a comma:

In [ ]:
btv_sea_flights_fall <- flights %>% 
    filter(origin == "JFK", (dest == "BTV" | dest == "SEA"), month >= 10)
View(btv_sea_flights_fall)

- Filter rows corresponding to flights that didn't go to Burlington, VT or Seattle, WA:

In [ ]:
not_BTV_SEA <- flights %>% 
    filter(!(dest == "BTV" | dest == "SEA"))
View(not_BTV_SEA)

- Note the careful use of parentheses around the `(dest == "BTV" | dest == "SEA")`
- If we didn't use parentheses, as follows:

In [ ]:
flights %>% filter(!dest == "BTV" | dest == "SEA")

- Returning all flights not headed to `"BTV"` *or* those headed to `"SEA"` 
- A larger number of airports one want to filter for, say `"SEA"`, `"SFO"`, `"PDX"`, `"BTV"`, and `"BDL"`
  - Continue to use the `|` (*or*) operator:

In [ ]:
many_airports <- flights %>% 
    filter(dest == "SEA" | dest == "SFO" | dest == "PDX" | 
           dest == "BTV" | dest == "BDL")

- This will get unwieldy to write as we progressively include more airports
  - A slightly shorter approach uses the `%in%` operator along with the `c()` function

In [ ]:
many_airports <- flights %>% 
    filter(dest %in% c("SEA", "SFO", "PDX", "BTV", "BDL"))
View(many_airports)

- `filter()` is recommended to be among the first verbs one consider applying to the data
  - It narrows down the scope of the data frame to just the observations one care about 

### Learning Check

What's another way using the "not" operator `!` to filter only the rows that are not going to Burlington, VT nor Seattle, WA in the `flights` data frame? Test this out using the code above.

**Solution**:

In [ ]:
# Original
not_BTV_SEA <- flights %>%
    filter(!(dest == "BTV" | dest == "SEA"))

# Alternative way
not_BTV_SEA <- flights %>%
    filter(!dest == "BTV" & !dest == "SEA")

# Yet another way
not_BTV_SEA <- flights %>%
    filter(dest != "BTV" & dest != "SEA")

## `summarize` variables 

- Summary statistics are single numerical values that summarize a large number of values
  - Commonly known examples of summary statistics
    - The mean (AKA the average)
    - The median (the middle value)
    - The sum
    - The smallest value (AKA the minimum), the largest value (AKA the maximum)
    - The standard deviation

- Example: calculate two summary statistics of the `temp` variable in the `weather` data frame -- the mean and the standard deviation
  - The `mean()` and `sd()` *summary functions* in R are needed 

![Diagram illustrating a summary function in R](./fig/summary.png)

- The `summarize()` function takes in a data frame and returns a data frame with only one row corresponding to the summary statistics 
  - Save the results in a new data frame `summary_temp` that will have two columns/variables: the `mean` and the `std_dev`

![Diagram of `summarize()` rows](./fig/summarize1.png)

In [ ]:
(summary_temp <- weather %>% 
    summarize(mean = mean(temp), 
	      std_dev = sd(temp)))

- Set `na.rm` to `TRUE`, where `rm` is short for "remove", to ignore any `NA` missing values and only return the summary value for all non-missing values:

In [ ]:
(summary_temp <- weather %>% 
  summarize(mean = mean(temp, na.rm = TRUE), 
            std_dev = sd(temp, na.rm = TRUE)))

- R does not ignore rows with missing values by default
- One should be mindful of the missingness and any potential causes of this missingness throughout the analysis

What are other summary functions we can use ? 
- One can use any function in R that takes many values and returns just one inside the `summarize()` verb to compute summary statistics
- Some selected functions
  - `mean()`: the average
  - `sd()`: the standard deviation, which is a measure of spread
  - `min()` and `max()`: the minimum and maximum values, respectively
  - `IQR()`: interquartile range
  - `sum()`: the total amount when adding multiple numbers
  - `n()`: a count of the number of rows in each group -- this particular summary function will make more sense when `group_by()` is covered

### Learning Check

Say a doctor is studying the effect of smoking on lung cancer for a large number of patients who have records measured at five year intervals. She notices that a large number of patients have missing data points because the patient has died, so she chooses to ignore these patients in her analysis. What is wrong with this doctor's approach?

**Solution**: The missing patients may have died of lung cancer! So to ignore them might seriously **bias** your results! It is very important to think of what the consequences on your analysis are of ignoring missing data! Ask yourself:

* There is a systematic reasons why certain values are missing? If so, you might be biasing your results!
* If there isn't, then it might be ok to "sweep missing values under the rug."

******

Modify the above `summarize` function to create `summary_temp` to also use the `n()` summary function: `summarize(count = n())`. What does the returned value correspond to?

**Solution**: It corresponds to a count of the number of observations/rows:

In [ ]:
weather %>%
  summarize(count = n())

******

Why doesn't the following code work?  Run the code line by line instead of all at once, and then look at the data.  In other words, run `summary_temp <- weather %>% summarize(mean = mean(temp, na.rm = TRUE))` first.

In [ ]:
summary_temp <- weather %>%
    summarize(mean = mean(temp, na.rm = TRUE)) %>%
    summarize(std_dev = sd(temp, na.rm = TRUE))

**Solution**: Consider the output of only running the first two lines:

In [ ]:
weather %>%
    summarize(mean = mean(temp, na.rm = TRUE))

Because after the first `summarize()`, the variable `temp` disappears as it has been collapsed to the value `mean`. So when we try to run the second `summarize()`, it can't find the variable `temp` to compute the standard deviation of. 

## `group_by` rows

![Diagram of `group_by()` and `summarize()`.](./fig/group_summary.png)

- Say one would like 12 mean temperatures for each of the 12 months separately
  - One can do this by "grouping" temperature observations by the 12 values of the variable `month`:

In [ ]:
(summary_monthly_temp <- weather %>% 
    group_by(month) %>% 
    summarize(mean = mean(temp, na.rm = TRUE), 
              std_dev = sd(temp, na.rm = TRUE)))

- Important: `group_by()` doesn't change data frames by itself -- it changes the *meta-data*, specifically the grouping structure 
- For examples, consider the operations to `diamonds` data frame included in `ggplot2`:

In [ ]:
diamonds %>% 
    group_by(cut)

diamonds %>% 
    group_by(cut) %>% 
    summarize(avg_price = mean(price))

- To remove this grouping structure meta-data, one can pipe the resulting data frame into the `ungroup()` function

In [ ]:
diamonds %>% 
    group_by(cut) %>% 
    ungroup()

- Suppose we'd like to count how many flights departed each of the three airports in New York City:

In [ ]:
(by_origin <- flights %>% 
    group_by(origin) %>% 
    summarize(count = n()))

- Newark (`"EWR"`) had the most flights departing in 2013 followed by `"JFK"` and lastly by LaGuardia (`"LGA"`)
- `sum()` returns the sum of a numerical variable, `n()` returns a count of the number of rows/observations 

### Grouping by more than one variable

- Say one want to know the number of flights leaving each of the three New York City airports *for each month*
  - Group by a second variable `month` using `group_by(origin, month)`

In [ ]:
(by_origin_monthly <- flights %>% 
    group_by(origin, month) %>% 
    summarize(count = n()))

- Why `group_by(origin, month)` and not `group_by(origin)` and then `group_by(month)`? 
  - Investigation

In [ ]:
(by_origin_monthly_incorrect <- flights %>% 
    group_by(origin) %>% 
    group_by(month) %>% 
    summarize(count = n()))

- `group_by(month)` overwrote the grouping structure meta-data of the earlier `group_by(origin)`
  - In the end only grouping by `month`
  - When `group_by()` two or more variables, one should include all the variables at the same time seperated by commas

### Learning Check

We looked at plots of temperatures by months in NYC (`weather` in `nycflights13`). What does the standard deviation column in the `summary_monthly_temp` data frame tell us about temperatures in NYC throughout the year?

**Solution**:

In [ ]:
summary_temp_by_month <- weather %>%
  group_by(month) %>%
  summarize(mean = mean(temp, na.rm = TRUE),
            std_dev = sd(temp, na.rm = TRUE))

kable(summary_temp_by_month) %>%
    kable_styling(font_size = 16)

******

The standard deviation is a quantification of **spread** and **variability**. We see that the period in November, December, and January has the most variation in weather, so you can expect very different temperatures on different days. What code would be required to get the mean and standard deviation temperature for each day in 2013 for NYC?

**Solution**:

In [ ]:
(summary_temp_by_day <- weather %>%
    group_by(year, month, day) %>%
    summarize(mean = mean(temp, na.rm = TRUE),
              std_dev = sd(temp, na.rm = TRUE)))

Note: `group_by(day)` is not enough, because `day` is a value between 1-31. We need to `group_by(year, month, day)` 

******

Recreate `by_monthly_origin`, but instead of grouping via `group_by(origin, month)`, group variables in a different order `group_by(month, origin)`. What differs in the resulting dataset?

**Solution**:

In [ ]:
(by_monthly_origin <- flights %>%
    group_by(month, origin) %>%
    summarize(count = n()))

In `by_monthly_origin` the `month` column is now first and the rows are sorted by `month` instead of origin. If you compare the values of `count` in `by_origin_monthly` and `by_monthly_origin` using the `View()` function, you'll see that the values are actually the same, just presented in a different order. 

******

How could we identify how many flights left each of the three airports for each `carrier`?

**Solution**: We could summarize the count from each airport using the `n()` function, which *counts rows*.

In [ ]:
(count_flights_by_airport <- flights %>%
    group_by(origin, carrier) %>%
    summarize(count = n()))

All remarkably similar! Note: the `n()` function counts rows, whereas the `sum(VARIABLE_NAME)` function sums all values of a certain numerical variable `VARIABLE_NAME`.

******

How does the `filter` operation differ from a `group_by` followed by a `summarize`?

**Solution**: 

* `filter` picks out rows from the original dataset without modifying them, whereas
* `group_by %>% summarize` computes summaries of numerical variables, and hence reports new values.

## `mutate`: create new variables based on existing ones

![Diagram of `mutate()` columns](./fig/mutate.png)

- Say you are more comfortable thinking of temperature in degrees Celsius instead of degrees Fahrenheit
- The formula to convert temperatures from &deg;F to &deg;C is $$\text{temp in C} = \frac{\text{temp in F} - 32}{1.8}$$
- One can apply the formula to the `temp` variable using the `mutate()` function which takes `temp` and create new variable `temp_in_C`

In [ ]:
weather <- weather %>% 
    mutate(temp_in_C = (temp - 32) / 1.8)

- Compute monthly average temperatures in both &deg;F and &deg;C using `group_by()` and `summarize()`:

In [ ]:
(summary_monthly_temp <- weather %>% 
    group_by(month) %>% 
    summarize(mean_temp_in_F = mean(temp, na.rm = TRUE), 
              mean_temp_in_C = mean(temp_in_C, na.rm = TRUE)))

- Create a variable known in the airline industry as _gain_ by `mutate()`:

In [ ]:
flights <- flights %>% 
    mutate(gain = dep_delay - arr_delay)

- Take a look at only the `dep_delay`, `arr_delay`, and the resulting `gain` variables for the first 5 rows in our updated `flights` data frame in Table .

In [ ]:
flights %>% 
    select(dep_delay, arr_delay, gain) %>% 
    slice(1:5) %>% 
    kable(caption = "First five rows of departure/arrival delay and gain variables") %>%
    kable_styling(position = "center") %>%
    as.character() %>%
    display_html()

- The flight in the first row departed 2 minutes late but arrived 11 minutes late
  - Its "gained time in the air" is a loss of 9 minutes, hence its `gain` is 2 $-$ 11 $=$ -9 
- The flight in the fourth row departed a minute early (`dep_delay` of -1) but arrived $18$ minutes early (`arr_delay` of -18)
  - Its "gained time in the air" is -1 $-$ (-18) $=$ -1 $+$ 18 $=$ 17 minutes, hence its `gain` is +17.
- Look at some summary statistics of the `gain` variable by considering multiple summary functions at once in the same `summarize()` code

In [ ]:
(gain_summary <- flights %>% 
    summarize(min = min(gain, na.rm = TRUE),
              q1 = quantile(gain, 0.25, na.rm = TRUE),
              median = quantile(gain, 0.5, na.rm = TRUE),
              q3 = quantile(gain, 0.75, na.rm = TRUE),
              max = max(gain, na.rm = TRUE),
              mean = mean(gain, na.rm = TRUE),
              sd = sd(gain, na.rm = TRUE),
              missing = sum(is.na(gain))))

- One can create multiple new variables at once in the same `mutate()` code

In [ ]:
flights <- flights %>% 
    mutate(gain = dep_delay - arr_delay,
           hours = air_time / 60,
           gain_per_hour = gain / hours)

### Learning Check

Could we create the `dep_delay` and `arr_delay` columns by simply subtracting `dep_time` from `sched_dep_time` and similarly for arrivals?  Try the code out and explain any differences between the result and what actually appears in `flights`.

**Solution**: No because you can't do direct arithmetic on times. The difference in time between 12:03 and 11:59 is 4 minutes, but 1203$-$ 1159 $=$ 44

## `arrange` and sort rows 

- Suppose we are interested in determining the most frequent destination airports for all domestic flights departing from New York City in 2013

In [ ]:
(freq_dest <- flights %>% 
    group_by(dest) %>% 
    summarize(num_flights = n()))

- Observe that by default the rows of the resulting `freq_dest` data frame are sorted in alphabetical order of `dest`ination
  - Say we would like to see the same data but sorted from the most to the least number of flights (`num_flights`) instead:

In [ ]:
(freq_dest %>% 
    arrange(num_flights))

- `arrange()` always returns rows sorted in ascending order by default 
  - To switch the ordering to be in "descending" order, one use the `desc()` function

In [ ]:
(freq_dest %>% 
    arrange(desc(num_flights)))

## `join` data frames

- In `flights` data frame, the variable `carrier` lists the carrier code for the different flights
  - This information is provided in a separate data frame `airlines`

In [ ]:
View(airlines)

- In `airlines`, `carrier` is the carrier code, while `name` is the full name of the airline company 
- Can one have all this information in a single data frame instead of two separate data frames?
  - One can do this by "joining" `flights` and `airlines` data frames
  - The values in the variable `carrier` in the `flights` data frame match the values in the variable `carrier` in the `airlines` data frame 

![`nycflights13` internal relations](./fig/relational-nycflights.png)


### Matching "key" variable names

- In `flights` and `airlines` data frames, the key variable to join/merge/match the rows by has the same name: `carrier`
- Use the `inner_join()` function to join the two data frames, where the rows will be matched by the variable `carrier`, and then compare the resulting data frames

In [ ]:
flights_joined <- flights %>% 
    inner_join(airlines, by = "carrier")
View(flights)
View(flights_joined)

- A visual representation of the `inner_join()` is shown 

![Inner Join.](./fig/join-inner.png)

- Other types of joins such as `left_join()`, `right_join()`, `outer_join()`, and `anti_join()` are available, but the `inner_join()` will solve the majority of problems

### Different "key" variable names

- The `airports` data frame contains the airport codes for each airport:

In [ ]:
View(airports)

- However, in `airports` and `flights` data frames, the airport codes are in variables that have different names 
  - In `airports` the airport code is in `faa`
  - In `flights` the airport codes are in `origin` and `dest`
- In order to join `airports` and `flights` by airport code, the `inner_join()` operation should use `by = c("dest" = "faa")` when the key variable has different names

In [ ]:
flights_with_airport_names <- flights %>% 
    inner_join(airports, by = c("dest" = "faa"))
View(flights_with_airport_names)

- Compute the number of flights from NYC to each destination, but also includes information about each destination airport

In [ ]:
(named_dests <- flights %>%
    group_by(dest) %>%
    summarize(num_flights = n()) %>%
    arrange(desc(num_flights)) %>%
    inner_join(airports, by = c("dest" = "faa")) %>%
    rename(airport_name = name))

### Multiple "key" variables

- In order to join `flights` and `weather`, more than one key variable `year`, `month`, `day`, `hour`, and `origin` are needed
  - The combination of these 5 variables act to uniquely identify each observational unit in the `weather` data frame
- Specifying a *vector* of key variables to join by using `c()`

In [ ]:
flights_weather_joined <- flights %>%
    inner_join(weather, by = c("year", "month", "day", "hour", "origin"))
View(flights_weather_joined)

### Learning Check

When joining `flights` and `weather` (or, in other words, matching the hourly weather values with each flight), why do we need to join by all of `year`, `month`, `day`, `hour`, and `origin`, and not just `hour`?

**Solution**: Because `hour` is simply a value between 0 and 23; to identify a *specific* hour, we need to know which year, month, day and at which airport.

### Normal forms

- The data frames included in the `nycflights13` package are in a form that minimizes redundancy of data
  - The `flights` data frame only saves the `carrier` code of the airline company; it does not include the actual name of the airline
  - The first row of `flights` has `carrier` equal to `UA`, but it does not include the airline name of "United Air Lines Inc." 
- The names of the airline companies are included in the `name` variable of the `airlines` data frame 
  - To have the airline company name included in `flights`, join `flights` and `airlines`:

In [ ]:
joined_flights <- flights %>% 
    inner_join(airlines, by = "carrier")
View(joined_flights)

- *Normal forms* of data: The *key* variable(s) that the joins based on are *identification variables*, no redundancies
- Both `dplyr` and [SQL](https://en.wikipedia.org/wiki/SQL) use such *normal forms*, and they are similar

## Other verbs 

### `select` a subset of columns (variables)

![Diagram of select() columns.](./fig/select.png)

- Identify the names of variables in `flights` data frame:

In [ ]:
glimpse(flights)

- If one only need the `carrier` and `flight` variables, use `select()` to pick up these two:

In [ ]:
flights %>% 
    select(carrier, flight)

- `select()` makes it easier to explore large datasets
  - limit the scope to only those variables we care most about
  - make the dataset viewing more digestible
- `select()` also drop or de-select certain variables
  - deselect using the `-` sign

In [ ]:
(flights_no_year <- flights %>% select(-year))

- Another way of selecting columns/variables is by specifying a range of columns

In [ ]:
(flight_arr_times <- flights %>% 
	select(month:day, arr_time:sched_arr_time))

- The above will `select()` all columns between `month` and `day`, as well as between `arr_time` and `sched_arr_time`, and drop the rest 
- `select()` can also be used to reorder columns when used with the `everything()` helper function
  - For example, suppose we want `hour`, `minute`, and `time_hour` to appear immediately after `year`, `month`, and `day` while not discarding the rest of the variables: `everything()` will pick up all remainings

In [ ]:
flights_reorder <- flights %>% 
    select(year, month, day, hour, minute, time_hour, everything())
glimpse(flights_reorder)

- The helper functions `starts_with()`, `ends_with()`, and `contains()` can be used to select variables/columns that match conditions

In [ ]:
flights %>% select(starts_with("a"))
flights %>% select(ends_with("delay"))
flights %>% select(contains("time"))

### `rename` variables/columns to have new names

- Suppose we want to only focus on `dep_time` and `arr_time` and change `dep_time` and `arr_time` to be `departure_time` and `arrival_time` instead in the `flights_time` data frame:

In [ ]:
flights_time_new <- flights %>% 
    select(dep_time, arr_time) %>% 
    rename(departure_time = dep_time, arrival_time = arr_time)
glimpse(flights_time_new)

- `departure_time = dep_time` renames the `dep_time` variable to have the new name `departure_time`
  - assign a new variable `departure_time` to have the same values as `dep_time` and then *delete* the variable `dep_time`

### `top_n` values of a variable

- Using `top_n` with `n = 10` and `wt = num_flights` to indicate that we want the rows in `named_dests` corresponding to the top 10 values of `num_flights`

In [ ]:
(named_dests %>% 
    top_n(n = 10, wt = num_flights))

- `arrange()` these results in descending order of `num_flights`

In [ ]:
named_dests  %>% 
    top_n(n = 10, wt = num_flights) %>% 
    arrange(desc(num_flights))

### Learning Check

What are some ways to select all three of the `dest`, `air_time`, and `distance` variables from `flights`?  Give the code showing how to do this in at least three different ways.

**Solution**:

In [ ]:
# The regular way:
flights %>%
  select(dest, air_time, distance)

# Since they are sequential columns in the dataset
flights %>%
  select(dest:distance)

# Not as effective, by removing everything else
flights %>%
  select(
    -year, -month, -day, -dep_time, -sched_dep_time, -dep_delay, -arr_time,
    -sched_arr_time, -arr_delay, -carrier, -flight, -tailnum, -origin,
    -hour, -minute, -time_hour
  )

******

How could one use `starts_with`, `ends_with`, and `contains` to select columns from the `flights` data frame?  Provide three different examples in total: one for `starts_with`, one for `ends_with`, and one for `contains`.

**Solution**:

In [ ]:
# Anything that starts with "d"
flights %>%
  select(starts_with("d"))
# Anything related to delays:
flights %>%
  select(ends_with("delay"))
# Anything related to departures:
flights %>%
  select(contains("dep"))

******

Why might we want to use the `select` function on a data frame?

**Solution**: To narrow down the data frame, to make it easier to look at. Using `View()` for example.

******

Create a new data frame that shows the top 5 airports with the largest arrival delays from NYC in 2013.

**Solution**:

In [ ]:
(top_five <- flights %>%
    group_by(dest) %>%
    summarize(avg_delay = mean(arr_delay, na.rm = TRUE)) %>%
    arrange(desc(avg_delay)) %>%
    top_n(n = 5))

## Conclusion

### Summary table

In [ ]:
# The following Google Doc is published to CSV and loaded using read_csv():
# https://docs.google.com/spreadsheets/d/1nRkXfYMQiTj79c08xQPY0zkoJSpde3NC1w6DRhsWCss/edit#gid=0

ch4_scenarios <- "https://docs.google.com/spreadsheets/d/e/2PACX-1vRgwl1lugQA6zxzfB6_0hM5vBjXkU7cbUVYYXLcWeaRJ9HmvNXyCjzJCgiGW8HCe1kvjLCGYHf-BvYL/pub?gid=0&single=true&output=csv" %>%
    read_csv(na = "") #%>%
    #select(-X1)

ch4_scenarios %>%
    kable(caption = "Summary of data wrangling verbs",
          booktabs = TRUE, format = "html") %>%
    as.character() %>%
    display_html()

### Cheatsheet 

- Check out RStudio's "Data Transformation with dplyr" cheatsheet
  - Access the `dplyr` cheatsheet by going to the RStudio Menu Bar -> Help -> Cheatsheets -> "Data Transformation with dplyr."

### Learning Check

An airline industry measure of a passenger airline's capacity is the [available seat miles](https://en.wikipedia.org/wiki/Available_seat_miles), which is equal to the number of seats available multiplied by the number of miles or kilometers flown summed over all flights. 

For example, let's consider the scenario in Figure below. Since the airplane has 4 seats and it travels 200 miles, the available seat miles are 4 $\times$ 200 $=$ 800.

![Example of available seat miles for one flight.](./fig/asm.png)

Extending this idea, let's say an airline had 2 flights using a plane with 10 seats that flew 500 miles and 3 flights using a plane with 20 seats that flew 1000 miles, the available seat miles would be 2 $\times$ 10 $\times$ 500 $+$ 3 $\times$ 20 $\times$ 1000 $=$ 70,000 seat miles. 

Using the datasets included in the `nycflights13` package, compute the available seat miles for each airline sorted in descending order. After completing all the necessary data wrangling steps, the resulting data frame should have 16 rows (one for each airline) and 2 columns (airline name and available seat miles). Here are some hints:

1. **Crucial**: Unless you are very confident in what you are doing, it is worthwhile to not starting coding right away, but rather first sketch out on paper all the necessary data wrangling steps not using exact code, but rather high-level *pseudocode* that is informal yet detailed enough to articulate what you are doing. This way you won't confuse *what* you are trying to do (the algorithm) with *how* you are going to do it (writing `dplyr` code). 
1. Take a close look at all the datasets using the `View()` function: `flights`, `weather`, `planes`, `airports`, and `airlines` to identify which variables are necessary to compute available seat miles.

**Solution**: Let's first display the entire solution.

In [ ]:
flights %>%
    inner_join(planes, by = "tailnum") %>%
    select(carrier, seats, distance) %>%
    mutate(ASM = seats * distance) %>%
    group_by(carrier) %>%
    summarize(ASM = sum(ASM, na.rm = TRUE)) %>%
    arrange(desc(ASM))

Let's now break this down step-by-step. To compute the available seat miles for a given flight, we need the `distance` variable from the `flights` data frame and the `seats` variable from the `planes` data frame, necessitating a join by the key variable `tailnum`. To keep the resulting data frame easy to view, we'll `select()` only these two variables and `carrier`:

In [ ]:
flights %>%
    inner_join(planes, by = "tailnum") %>%
    select(carrier, seats, distance)

Now for each flight we can compute the available seat miles `ASM` by multiplying the number of seats by the distance via a `mutate()`:

In [ ]:
flights %>%
    inner_join(planes, by = "tailnum") %>%
    select(carrier, seats, distance) %>%
    # Added:
    mutate(ASM = seats * distance)

Next we want to sum the `ASM` for each carrier. We achieve this by first grouping by `carrier` and then summarizing using the `sum()` function:

In [ ]:
flights %>%
    inner_join(planes, by = "tailnum") %>%
    select(carrier, seats, distance) %>%
    mutate(ASM = seats * distance) %>%
    # Added:
    group_by(carrier) %>%
    summarize(ASM = sum(ASM))

However, because for certain carriers certain flights have missing `NA` values, the resulting table also returns `NA`'s. We can eliminate these by adding a `na.rm = TRUE` argument to `sum()`, telling R that we want to remove the `NA`'s in the sum:

In [ ]:
flights %>%
    inner_join(planes, by = "tailnum") %>%
    select(carrier, seats, distance) %>%
    mutate(ASM = seats * distance) %>%
    group_by(carrier) %>%
    # Modified:
    summarize(ASM = sum(ASM, na.rm = TRUE))

Finally, we `arrange()` the data in `desc()`ending order of `ASM`.

In [ ]:
flights %>%
    inner_join(planes, by = "tailnum") %>%
    select(carrier, seats, distance) %>%
    mutate(ASM = seats * distance) %>%
    group_by(carrier) %>%
    summarize(ASM = sum(ASM, na.rm = TRUE)) %>%
    # Added:
    arrange(desc(ASM))

While the above data frame is correct, the IATA `carrier` code is not always useful. For example, what carrier is `WN`? We can address this by joining with the `airlines` dataset using `carrier` is the key variable. While this step is not absolutely required, it goes a long way to making the table easier to make sense of. It is important to be empathetic with the ultimate consumers of your presented data!

In [ ]:
flights %>%
    inner_join(planes, by = "tailnum") %>%
    select(carrier, seats, distance) %>%
    mutate(ASM = seats * distance) %>%
    group_by(carrier) %>%
    summarize(ASM = sum(ASM, na.rm = TRUE)) %>%
    arrange(desc(ASM)) %>%
    # Added:
    inner_join(airlines, by = "carrier")